# 0-1背包问题
- 有一个背包，它的容量为C(Capacity)。现在有n中不同的物品，编号为0……n-1， 其中每一件物品的重量为$w(i)$，价值为$v(i)$。问可以向这个背包中盛放哪些物品，是的在不超过背包容量的基础上，物品的总价值最大。

## 分析
- 状态定义，$F(n, C)$， 考虑将n个物品放进容量为C的背包，使得价值最大
- 状态转移方程：
    - $F(i,c) = F(i-1, c)$  # 新来一个物品，不放进来
    - $F(i,c) = v(i) + F(i-1, c-w(i))$   # 新来一个物品放进来
    - 以上两种方式取最大
    - $F(i,c) = max(F(i-1, c), v(i) + F(i-1, c-w(i)))$

In [1]:
import time
def countTime(func):
    def wrapper(V, W, idx, C):
        start_time = time.time()
        func(V, W, idx, C)
        count = time.time() - start_time
        print("耗时：{}s".format(count))
    return wrapper

In [2]:
import random
n_goods = 25   # 物品总数
test_C = 100
test_V = [random.randint(1, 8) for i in range(n_goods)]   # 价值数组
test_W = [random.randint(1, 8) for i in range(n_goods)]   # 重量数组
print("价值数组", test_V)
print("重量数组", test_W)

价值数组 [1, 5, 7, 5, 1, 7, 5, 2, 2, 1, 5, 4, 3, 2, 5, 6, 8, 1, 2, 8, 8, 4, 6, 1, 7]
重量数组 [5, 2, 6, 8, 7, 8, 5, 4, 4, 4, 5, 3, 6, 4, 4, 7, 2, 4, 8, 5, 6, 7, 2, 4, 5]


## 1.递归算法

In [3]:
def best_value1(V, W, idx, C):
    if C <= 0 or idx < 0:
        return 0
    res = best_value1(V, W, idx-1, C);
    if W[idx] <= C:
        res = max(res, V[idx] + best_value1(V, W, idx-1, C-W[idx]))
    return res

@countTime
def main(V, W, idx, C):
    print("result:", best_value1(V, W, idx-1, C))
    
main(test_V, test_W, n_goods, test_C)

result: 100
耗时：30.493993520736694s


## 2.记忆化搜索

In [4]:
import numpy as np

In [5]:
def best_value2(V, W, idx, C):
    if C <= 0 or idx < 0:
        return 0
    
    if memo[idx][C] != -1:
        return memo[idx][C]
    
    res = best_value2(V, W, idx-1, C);
    if W[idx] <= C:
        res = max(res, V[idx] + best_value2(V, W, idx-1, C-W[idx]))
    memo[idx][C] = res
    return res

@countTime
def main(V, W, idx, C):
    global memo
    memo = np.array([[-1] * (C+1)] * idx)
    print("result:", best_value2(V, W, idx-1, C))
    
main(test_V, test_W, n_goods, test_C)

result: 100
耗时：0.008697986602783203s


## 3.动态规划

In [6]:
def best_value3(V, W, C):
    assert len(V)==len(W), "value列表和weight列表不一样长"
    n = len(V)
    if n<=0:
        return 0
    memo = np.array([[-1] * (C+1)] * n)
    for j in range(0, C+1):
        memo[0][j] = (V[0] if j>= W[0] else 0)
        
    for i in range(1, n):
        for j in range(0, C+1):
            memo[i][j] = memo[i-1][j]
            if j>=W[i]:
                memo[i][j] = max(memo[i][j], V[i] + memo[i-1][j-W[i]])
    return memo[n-1][C]

@countTime
def main(test_V, test_W, n_goods, test_C):
    print("result:", best_value3(test_V, test_W, test_C))
    
main(test_V, test_W, n_goods, test_C)

result: 100
耗时：0.007692813873291016s


## Notice:
二维数组的初始化和数值替换可能有问题，就是因为这个问题，我特么debug了N长时间

In [7]:
n = 5
c = 10
a = [[-1] * (c+1)] * n
a

[[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]]

In [8]:
a[n-1][c] = 9
a

[[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 9],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 9],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 9],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 9],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 9]]

In [9]:
b = np.array([[-1] * (c+1)] * n)
b

array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])

In [10]:
b[n-1][c] = 9
b

array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1,  9]])